# 잔차 블록(Residual Block)
: 신경망의 깊이를 증가시켜주면서 학습 성능을 향상시킨다.(ResNet(잔차 네트워크)에서 사용되는 종류 중 하나)

- 입력을 그대로 전달하는 지름길을 추가하여 신경망이 깊어질 수록 발생할 수 있는 기울기 소실 문제를 완화시킨다.
- 입력 특징 맵과 합성곱 레이어, 스킵 연결(지름길), 출력으로 총 4가지의 형태를 갖춘다.(밑 코드 참고)
- y = F(x) + x, ( F(x) : 합성곱 층을 지나온 변환 함수 )
    - 입력 신호(x) : 원래 입력 데이터 또는 이진 잔차 블록의 출력
    - 합성곱 레이어(첫 번째 줄) : 일반적으로 ReLU 활성화 함수
    - 합성곱 레이어(두 번째 줄) ::
    - 스킵 연결(지름길) : 입력 신호를 직접 더하는 연결
    - 출력 신호 : 스킵 연결을 포함한 최종 출력

In [14]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(32, 32, 3)) # 입력층(입력 이미지 세팅)
x = layers.Conv2D(32, 3, activation='relu')(inputs) # 첫 번째 합성곱 레이어
residual = x # 현재 특징맵 residual에 저장
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x) # 두 번째 합성곱 레이어
residual = layers.Conv2D(64, 1)(residual) # 입력 특징 맵 차원을 조정하기 위해 1*1 합성곱 레이어를 적용한다.
x = layers.add([x, residual]) # 두번째 합성곱 레이어의 출력과 조정된 입력 특징맵을 더해서 잔파 연결을 수행한다.

In [15]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation='relu')(inputs)
residual = x
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2, padding='same')(x) # 풀링 크기 2*2
residual = layers.Conv2D(64, 1, strides = 2)(residual) # 풀링 크기에 맞춰 strides 설정
# 연결하기 전 사이즈 정도는 맞춰주자는 느낌
x = layers.add([x, residual])

In [16]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Rescaling(1./255)(inputs)

def residual_block(x, filters, pooling=False):
    residual = x # 입력 데이터를 잔차에 저장 
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x) # 기본 히든층 생성

    if pooling: # pooling이 True라면, MaxPooling <-> 채널 수를 '1'로 맞춰준다.
        x = layers.MaxPooling2D(2, padding='same')(x)
        residual = layers.Conv2D(filters, 1, strides=2)(residual)
    elif filters != residual.shape[-1]: # 채널 수가 다르다면
        residual = layers.Conv2D(filters, 1)(residual) # 채널 수 맞춤

    x = layers.add([x, residual]) # 잔차연결
    return x

In [17]:
x = residual_block(x, filters=32, pooling=True)
x = residual_block(x, filters=64, pooling=True)
x = residual_block(x, filters=128, pooling=False)

x = layers.GlobalAveragePooling2D()(x) # 전역 평균 풀링 레이어를 통해 특징 맵의 차원을 1로 줄인다.
outputs = layers.Dense(1, activation='sigmoid')(x) 
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 rescaling_2 (Rescaling)     (None, 32, 32, 3)            0         ['input_10[0][0]']            
                                                                                                  
 conv2d_40 (Conv2D)          (None, 32, 32, 32)           896       ['rescaling_2[0][0]']         
                                                                                                  
 conv2d_41 (Conv2D)          (None, 32, 32, 32)           9248      ['conv2d_40[0][0]']           
                                                                                            

In [10]:
import gdown
gdown.download(id='18uC7WTuEXKJDDxbj-Jq6EjzpFrgE7IAd', output='dogs-vs-cats.zip')

Downloading...
From (original): https://drive.google.com/uc?id=18uC7WTuEXKJDDxbj-Jq6EjzpFrgE7IAd
From (redirected): https://drive.google.com/uc?id=18uC7WTuEXKJDDxbj-Jq6EjzpFrgE7IAd&confirm=t&uuid=12d8a496-a058-4e78-8986-03f4af328f37
To: /Users/jeon-yewon/Desktop/데이터 분석 강의/부트캠프/8주차/06.17./dogs-vs-cats.zip
100%|████████████████████████████████████████| 852M/852M [01:30<00:00, 9.46MB/s]


'dogs-vs-cats.zip'

In [18]:
import zipfile

with zipfile.ZipFile('dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall()

with zipfile.ZipFile('train.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [26]:
import os, shutil, pathlib
from tensorflow.keras.utils import image_dataset_from_directory

original_dir = pathlib.Path('train')
new_base_dir = pathlib.Path('cats_vs_dags_small')

def make_subset(subset_name, start_index, end_index):
    for category in ('cat', 'dog'):
        dir = new_base_dir / subset_name / category
        if not dir.exists():
            os.makedirs(dir)
        fnames = [f'{category}.{i}.jpg' for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst = dir/fname)

In [27]:
make_subset('train', start_index=0, end_index=1000)
make_subset('validation', start_index=1000, end_index=1500)
make_subset('test', start_index=1500, end_index=2500)

In [28]:
train_dataset = image_dataset_from_directory(
    new_base_dir / 'train', 
    image_size = (180, 180),
    batch_size = 32)

validation_dataset = image_dataset_from_directory(
    new_base_dir / 'validation',
    image_size = (180, 180),
    batch_size = 32)

test_dataset = image_dataset_from_directory(
    new_base_dir / 'test',
    image_size = (180, 180),
    batch_size = 32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [29]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models

inputs = tf.keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)

x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32, 64, 128, 256, 512]:
    residual = x

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(size, 3, padding='same', use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(size, 3, padding='same', use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    residual = layers.Conv2D(size, 1, strides=2, padding='same', use_bias=False)(residual)
    x = layers.add([x, residual])

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

Epoch 1/10
63/63 [==============================] - 174s 3s/step - loss: 0.7172 - accuracy: 0.5725 - val_loss: 0.7026 - val_accuracy: 0.5000
Epoch 2/10
63/63 [==============================] - 155s 2s/step - loss: 0.6594 - accuracy: 0.5990 - val_loss: 0.6964 - val_accuracy: 0.5000
Epoch 3/10
63/63 [==============================] - 155s 2s/step - loss: 0.6341 - accuracy: 0.6390 - val_loss: 0.7087 - val_accuracy: 0.5000
Epoch 4/10
63/63 [==============================] - 170s 3s/step - loss: 0.6271 - accuracy: 0.6585 - val_loss: 0.7097 - val_accuracy: 0.5000
Epoch 5/10
63/63 [==============================] - 276s 4s/step - loss: 0.6011 - accuracy: 0.6885 - val_loss: 0.8045 - val_accuracy: 0.5000
Epoch 6/10
63/63 [==============================] - 283s 4s/step - loss: 0.5807 - accuracy: 0.6965 - val_loss: 0.8330 - val_accuracy: 0.5020
Epoch 7/10
63/63 [==============================] - 283s 4s/step - loss: 0.5722 - accuracy: 0.7165 - val_loss: 0.8358 - val_accuracy: 0.5300
Epoch 8/10
63

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accuracy)+1)
plt.plot(epochs, accuracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training Accuracy')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()